**Ganti Runtime ke GPU sebelum menjalankan kode**

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Seleksi Model Menggunakan Pre-trained Model Yolov8
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
%cd {HOME}
Image(filename='runs/detect/predict/dog.jpeg', height=600)

**Sesuaikan dengan database dari Roboflow**

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets
# Pengumpulan Dataset dari Roboflow (sesuaikan dengan dataset masing2)
!pip install roboflow --quiet
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("").project("")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
# Training atau pelatihan model Yolov8 ke dataset yang sudah disiapkan dari Roboflow
%cd {HOME}
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=20 imgsz=320 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

In [ ]:
# Evalusai Model (Proses Validasi)
%cd {HOME}
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
# Evaluasi Model (Proses Testing)
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cp -av "/content/runs" "/content/drive/MyDrive/YoloApel2"

**Disconnect Runtime dan ganti ke CPU**

In [ ]:
!pip install ultralytics

In [ ]:
import shutil
shutil.copy("/content/drive/MyDrive/YoloApel2/runs/detect/train/weights/best.pt", "/content")

In [ ]:
# Deployment Model Ke Android (Konversi model menjadi format Tensorflow Lite)
!yolo export model=/content/best.pt format=tflite